In [1]:
using Flux, ParameterSchedulers, Optimisers, Statistics, CUDA 
using Flux: Conv, BatchNorm, MaxPool, flatten, Dense, Dropout, relu, softmax
using Base.Iterators: repeated, partition
using Printf, BSON, JLD2, LinearAlgebra
using ImageCore, Images
using MLDatasets: convert2image, CIFAR10
using Plots              
using StatsBase

┌ Warning: Package cuDNN not found in current path.
│ - Run `import Pkg; Pkg.add("cuDNN")` to install the cuDNN package, then restart julia.
│ - If cuDNN is not installed, some Flux functionalities will not be available when running on the GPU.
└ @ FluxCUDAExt C:\Users\oliwi\.julia\packages\Flux\9PibT\ext\FluxCUDAExt\FluxCUDAExt.jl:10


In [2]:
ENV["DATADEPS_ALWAYS_ACCEPT"] = "true"

"true"

In [3]:
use_cuda = true

true

In [4]:
if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

┌ Info: Training on CPU
└ @ Main c:\Users\oliwi\Desktop\SGH\masgisterka\II_sem\Deep Learning\projekt_zaliczeniowy\DeepLearningProjektZaliczeniowy\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:6


In [5]:
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, 32, 32, 3, length(idxs))
    for (i, idx) in enumerate(idxs)
        X_batch[:,:,:,i] = X[:,:,:,idx]
    end
    Y_batch = Flux.onehotbatch(Y[idxs], 0:9)
    return X_batch, Y_batch
end

make_minibatch (generic function with 1 method)

In [6]:
test_set = CIFAR10(:test)
test_set = make_minibatch(test_set.features, test_set.targets, 1:size(test_set.features)[4]) |> device

(Float32[0.61960787 0.59607846 … 0.23921569 0.21176471; 0.62352943 0.5921569 … 0.19215687 0.21960784; … ; 0.49411765 0.49019608 … 0.11372549 0.13333334; 0.45490196 0.46666667 … 0.078431375 0.08235294;;; 0.4392157 0.4392157 … 0.45490196 0.41960785; 0.43529412 0.43137255 … 0.4 0.4117647; … ; 0.35686275 0.35686275 … 0.32156864 0.32941177; 0.33333334 0.34509805 … 0.2509804 0.2627451;;; 0.19215687 0.2 … 0.65882355 0.627451; 0.18431373 0.15686275 … 0.5803922 0.58431375; … ; 0.14117648 0.1254902 … 0.49411765 0.5058824; 0.12941177 0.13333334 … 0.41960785 0.43137255;;;; 0.92156863 0.93333334 … 0.32156864 0.33333334; 0.90588236 0.92156863 … 0.18039216 0.24313726; … ; 0.9137255 0.9254902 … 0.7254902 0.7058824; 0.9098039 0.92156863 … 0.73333335 0.7294118;;; 0.92156863 0.93333334 … 0.3764706 0.39607844; 0.90588236 0.92156863 … 0.22352941 0.29411766; … ; 0.9137255 0.9254902 … 0.78431374 0.7647059; 0.9098039 0.92156863 … 0.7921569 0.78431374;;; 0.92156863 0.93333334 … 0.32156864 0.3254902; 0.90588236

In [7]:
model_small_convnet = Chain(
  # — blok 1: 32×32 → 16×16, kanały 3 → 16 —
  Conv((3,3), 3=>16, pad=1), BatchNorm(16), relu,
  MaxPool((2,2)),                           # 16×16

  # — blok 2: 16×16 → 8×8, kanały 16 → 32 —
  Conv((3,3), 16=>32, pad=1), BatchNorm(32), relu,
  MaxPool((2,2)),                           # 8×8

  # — blok 3: 8×8 → 4×4, kanały 32 → 64 —
  Conv((3,3), 32=>64, pad=1), BatchNorm(64), relu,
  MaxPool((2,2)),                           # 4×4

  # — flatten + head FC —
  flatten,                                  # 64*4*4 = 1024
  Dense(1024, 128), relu, Dropout(0.5),
  Dense(128,   10),
  softmax
) |> device

BSON.@load "model_small_convnet.bson" ps

Flux.loadmodel!(model_small_convnet, device(ps))

Chain(
  Conv((3, 3), 3 => 16, pad=1),         # 448 parameters
  BatchNorm(16),                        # 32 parameters, plus 32
  NNlib.relu,
  MaxPool((2, 2)),
  Conv((3, 3), 16 => 32, pad=1),        # 4_640 parameters
  BatchNorm(32),                        # 64 parameters, plus 64
  NNlib.relu,
  MaxPool((2, 2)),
  Conv((3, 3), 32 => 64, pad=1),        # 18_496 parameters
  BatchNorm(64),                        # 128 parameters, plus 128
  NNlib.relu,
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(1024 => 128),                   # 131_200 parameters
  NNlib.relu,
  Dropout(0.5),
  Dense(128 => 10),                     # 1_290 parameters
  NNlib.softmax,
)         # Total: 16 trainable arrays, 156_298 parameters,
          # plus 6 non-trainable, 224 parameters, summarysize 613.047 KiB.

In [8]:
X_test, Y_test = test_set 

(Float32[0.61960787 0.59607846 … 0.23921569 0.21176471; 0.62352943 0.5921569 … 0.19215687 0.21960784; … ; 0.49411765 0.49019608 … 0.11372549 0.13333334; 0.45490196 0.46666667 … 0.078431375 0.08235294;;; 0.4392157 0.4392157 … 0.45490196 0.41960785; 0.43529412 0.43137255 … 0.4 0.4117647; … ; 0.35686275 0.35686275 … 0.32156864 0.32941177; 0.33333334 0.34509805 … 0.2509804 0.2627451;;; 0.19215687 0.2 … 0.65882355 0.627451; 0.18431373 0.15686275 … 0.5803922 0.58431375; … ; 0.14117648 0.1254902 … 0.49411765 0.5058824; 0.12941177 0.13333334 … 0.41960785 0.43137255;;;; 0.92156863 0.93333334 … 0.32156864 0.33333334; 0.90588236 0.92156863 … 0.18039216 0.24313726; … ; 0.9137255 0.9254902 … 0.7254902 0.7058824; 0.9098039 0.92156863 … 0.73333335 0.7294118;;; 0.92156863 0.93333334 … 0.3764706 0.39607844; 0.90588236 0.92156863 … 0.22352941 0.29411766; … ; 0.9137255 0.9254902 … 0.78431374 0.7647059; 0.9098039 0.92156863 … 0.7921569 0.78431374;;; 0.92156863 0.93333334 … 0.32156864 0.3254902; 0.90588236

In [9]:
function loss_batched(model, X, Y; batchsize=64)
  total_loss = 0f0
  total_n    = 0
  N = size(X, 4)
  for i in 1:batchsize:N
    j = min(i+batchsize-1, N)
    xb = device(X[:,:,:, i:j])
    yb = device(Y[:,   i:j])
    # 1) oblicz stratę na batchu
    l = Flux.crossentropy(model(xb), yb)
    # 2) sumuj stratę ważoną liczbą próbek
    nb = size(xb, 4)
    total_loss += l * nb
    total_n    += nb
  end
  return total_loss / total_n
end

loss_batched (generic function with 1 method)

In [ ]:
loss_batched(model_small_convnet, X_test, Y_test; batchsize=64)

OutOfMemoryError: OutOfMemoryError()

In [10]:
function accuracy_batched(model, X, Y; batchsize=64)
  total, correct = 0, 0
  N = size(X, 4)
  for i in 1:batchsize:N
    j = min(i+batchsize-1, N)
    xb = device(X[:,:,:, i:j])
    yb = device(Y[:,   i:j])
    ŷ = model(xb)
    p = Flux.onecold(ŷ, 0:9)
    t = Flux.onecold(yb, 0:9)
    correct += sum(p .== t)
    total   += length(t)
  end
  return correct / total
end

accuracy_batched (generic function with 1 method)

In [ ]:
accuracy_batched(model_small_convnet, X_test, Y_test; batchsize=64)

In [11]:
# 1) Predykcje i prawdziwe etykiety jako wektory 0–9
preds = Flux.onecold(model_small_convnet(X_test), 0:9)
trues = Flux.onecold(Y_test,      0:9)

# 2) Liczniki
class_correct = zeros(Int, 10)
class_total   = zeros(Int, 10)

# 3) Pętla po wszystkich próbkach
for (p, t) in zip(preds, trues)
    class_total[t+1]   += 1        
    class_correct[t+1] += (p == t)
end

# 4) Accuracy 
class_accuracy = class_correct ./ class_total
for i in 1:10
    @printf("%-6s: %5.2f%%  (%4d/%4d)\n",
        classes[i], 100*class_accuracy[i],
        class_correct[i], class_total[i])
end

OutOfMemoryError: OutOfMemoryError()

In [12]:
using Images, ImageShow, Plots

preds = Flux.onecold(model_small_convnet(X_test), 0:9)
trues = Flux.onecold(Y_test,      0:9)

function show_predictions(X, trues, preds; n=10)
    idxs = rand(1:size(X,4), n)
    cols = min(n,5); rows = ceil(Int,n/cols)
    plt = plot(layout=(rows,cols), margin=2Plots.mm, xticks=false, yticks=false)
    for (i,j) in enumerate(idxs)
        raw = X[:,:,:,j] |> cpu
        chw = permutedims(raw, (3,2,1))
        img = collect(colorview(RGB, chw))
        title = "$(classes[trues[j]+1]) → $(classes[preds[j]+1])"
        plot!(plt[i], img, seriestype=:heatmap,
              aspect_ratio=1, axis=false, title=title, titlefontsize=8)
    end
    display(plt)
end

show_predictions(X_test, trues, preds; n=15)

OutOfMemoryError: OutOfMemoryError()

In [ ]:
using StatsBase, Plots

cm = zeros(Int, 10, 10)
for (t,p) in zip(trues, preds)
    cm[t+1, p+1] += 1
end

heatmap(
  cm;
  xticks=(1:10, classes),
  yticks=(1:10, classes),
  xlabel="Predicted",
  ylabel="Actual",
  title="Confusion Matrix",
  color = :blues,                # paleta
  clims = (0, maximum(cm)),      # skala kolorów od 0 do max
  aspect_ratio = 1,              # kwadratowe komórki
  right_margin = 10Plots.mm,
  annotate = cm                  # wstawia wartości cm[i,j] w każdej komórce
)


In [ ]:
# using StatsBase, Plots

# cm = zeros(Int, 10, 10)
# for (t,p) in zip(trues, preds)
#     cm[t+1, p+1] += 1
# end

# heatmap(
#   cm,
#   xticks=(1:10, classes),
#   yticks=(1:10, classes),
#   xlabel="Predicted",
#   ylabel="Actual",
#   title="Confusion Matrix",
#   right_margin = 10Plots.mm,
# )